https://cloud.google.com/blog/ja/products/gcp/my-summer-project-a-rock-paper-scissors-machine-built-on-tensorflow　
こちらのサイトを参考にしました。

In [20]:
# import pressure sensor data from CSV files
from __future__ import print_function
import numpy as np
p_data_wu = np.genfromtxt("wu_pressuresensor_data.csv", dtype=np.float32, delimiter=",")
p_data_takeuchi = np.genfromtxt("takeuchi_pressuresensor_data.csv", dtype=np.float32, delimiter=",")
p_data_zero = np.genfromtxt("zero_pressuresensor_data.csv", dtype=np.float32, delimiter=",")
p_data = np.append(p_data_wu, p_data_takeuchi, axis=0)
p_data = np.append(p_data, p_data_zero, axis=0)

# create boarder line
boarder1 = np.sum(p_data[0:1999]) / 2000 
boarder2 = np.sum(p_data[1000:2999]) / 2000
print("boarder1: " + str(boarder1) + ", boarder2: " + str(boarder2))

# create conversion data
p_data_wu = list(map(lambda x: 2 if x >= boarder1 else (1 if x >= boarder2 else 0), p_data_wu))
p_data_takeuchi = list(map(lambda x: 2 if x >= boarder1 else (1 if x >= boarder2 else 0), p_data_takeuchi))
p_data_zero = list(map(lambda x: 2 if x >= boarder1 else (1 if x >= boarder2 else 0), p_data_zero))

# merge all to p_data
p_data_wu = np.reshape(p_data_wu, (1000, 1))
p_data_takeuchi = np.reshape(p_data_takeuchi, (1000, 1))
p_data_zero = np.reshape(p_data_zero, (1000, 1))
p_data = np.append(p_data_wu, p_data_takeuchi, axis=0)
p_data = np.append(p_data, p_data_zero, axis=0)
p_data = np.float32(p_data)
print("pressure sensor data: \n" + str(p_data))

boarder1: 750.6205, boarder2: 296.263
pressure sensor data: 
[[2.]
 [2.]
 [2.]
 ...
 [0.]
 [0.]
 [0.]]


In [21]:
# create coaching data
p_label_wu = np.tile(2, (p_data_wu.shape[0], 1))
p_label_takeuchi = np.tile(1, (p_data_takeuchi.shape[0], 1))
p_label_zero = np.tile(0, (p_data_zero.shape[0], 1))
p_label = np.append(p_label_wu, p_label_takeuchi, axis=0)
p_label = np.append(p_label, p_label_zero, axis=0)
print("pressure sensor label: \n" + str(p_label))

pressure sensor label: 
[[2]
 [2]
 [2]
 ...
 [0]
 [0]
 [0]]


In [22]:
# define the tensoflow and weight function
import tensorflow as tf
weights = tf.Variable(tf.zeros([1, 3]), dtype=tf.float32)
biases = tf.Variable(tf.zeros([3]), dtype=tf.float32)
pressure_sensor_data = tf.placeholder(tf.float32)
rps_data = tf.matmul(p_data, weights) + biases

# coaching
rps_labels = tf.placeholder(tf.int32)
loss = tf.losses.sparse_softmax_cross_entropy(labels=rps_labels, logits=rps_data)

# define an optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1).minimize(loss)

# create a Session
sess = tf.Session()

# init the variables
initializer = tf.global_variables_initializer()
sess.run(initializer)

# train the model
for i in range(10000): 
  sess.run(optimizer, {pressure_sensor_data: p_data, rps_labels: p_label})
  
# print current loss
  if i % 1000 == 0:
    l = sess.run(loss, {pressure_sensor_data: p_data, rps_labels: p_label})
    print("steps = {}, loss = {:f}".format(i, l))

steps = 0, loss = 0.936285
steps = 1000, loss = 0.017387
steps = 2000, loss = 0.008568
steps = 3000, loss = 0.005674
steps = 4000, loss = 0.004239
steps = 5000, loss = 0.003382
steps = 6000, loss = 0.002813
steps = 7000, loss = 0.002408
steps = 8000, loss = 0.002105
steps = 9000, loss = 0.001869


In [25]:
# print weights
print("float weights[1][3] = {")
for v in sess.run(weights):
  print("  {{{:f},{:f},{:f}}},".format(v[0], v[1], v[2]))
print("};\n")

# print biases
v = sess.run(biases)
print("float biases[3] = {{{:f},{:f},{:f}}};\n".format(v[0], v[1], v[2]))

float weights[1][3] = {
  {-13.754920,0.457206,13.297680},
};

float biases[3] = {10.884060,4.080285,-14.964365};

